# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])




In [75]:
#Calculate Total number of schools
total_schools = school_data["school_name"].count()
#Calculate the total budget of each school
total_budget = school_data["budget"].sum()
#Calculate the total number of students at each school
total_students = student_data["student_name"].count()
#Caclulate the average student math score
avg_math = round(student_data["math_score"].mean(),1)
#Calculate the average student reading score
avg_reading = student_data["reading_score"].mean()
#calculate the overall average math and reading score
total_average =(avg_reading + avg_math) / 2

#calculate the number of passing math students
passing_math_students = school_data_complete.loc[school_data_complete["math_score"] >= 70]
#calculate the percent passing math students
passing_math_percent = (passing_math_students["math_score"].count() / total_students)*100
#calculate the number of passing reading students
passing_reading_students = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
#calculate the percent passing reading students
passing_reading_percent = (passing_reading_students["reading_score"].count() *100) / total_students




In [70]:
#Create dataframe with from the series above
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                           "Total Students": [total_students],
                           "Total Budget": [total_budget],
                           "Average Math Score": [avg_math],
                           "Average Reading Score": [avg_reading],
                           "% Passing Math": [passing_math_percent],
                           "% Passing Reading": [passing_reading_percent],
                           "% Overall Passing Rate": [total_average]})                   

#edit format to add commas and dollar sign to budget numbers
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

#print data frame
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",79.0,81.87784,74.980853,85.805463,80.43892


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [54]:
#Group schools by name
school_data_grouped = school_data_complete.groupby('school_name')
#calculate the avearages, totals, and counts of school information
school_data_averages = school_data_grouped.mean()
school_data_count = school_data_grouped.sum()
school_data_sum = school_data_grouped.count()

#merge the averages, sums, and counts using the school name as key
school_data_merge = pd.merge(school_data_averages, school_data_count, how="outer", on="school_name")
school_data_merge = pd.merge(school_data_merge, school_data_sum, how="outer", on="school_name")

#Group student information by school and calculate averages, sums, and counts of various parameters
math_passing_students = passing_math_students.groupby('school_name')
sum_passing_math = math_passing_students.count()
reading_passing_students = passing_reading_students.groupby('school_name')
sum_passing_reading = reading_passing_students.count()

#merge all of the calculated data frames
total_passing_merge = pd.merge(sum_passing_math, sum_passing_reading, how="outer", on="school_name")
tot_passing_size = pd.merge(total_passing_merge, school_data_averages, how="outer", on="school_name")
tot_pass_reduced = tot_passing_size[["size_x", "size_y", "size"]]
tot_pass_reduced = tot_pass_reduced.rename(columns={"size_x": "tot_passing_math", "size_y": "tot_passing_reading",
                                                   "size": "total_students"})
tot_pass_reduced["math_%"] = (tot_pass_reduced["tot_passing_math"] / tot_pass_reduced["total_students"]) * 100 
tot_pass_reduced["reading_%"] = (tot_pass_reduced["tot_passing_reading"] / tot_pass_reduced["total_students"]) * 100
tot_pass_reduced["avg_pass"] = (tot_pass_reduced["math_%"] + tot_pass_reduced["reading_%"]) / 2                                           
reduced_df = school_data_merge[["size_x", "budget_x", "reading_score_x",
                       "math_score_x"]]

#Calculate the Per Student Budget
reduced_df["per_student_budget"] = reduced_df["budget_x"] / reduced_df["size_x"]

#More merging of dataframes
Student_school_summary = pd.merge(reduced_df, tot_pass_reduced, how="outer", on="school_name")
Student_school_summary = pd.merge(Student_school_summary, school_data, how="outer", on="school_name")


#Rename columns 
Student_school_summary = Student_school_summary.rename(columns={"school_name":"School Name","size_x":"Total Students", 
                                                                "budget_x": "Total School Budget",
                                                                "reading_score_x": "Average Reading Score",
                                                                "math_score_x":"Average Math Score",
                                                                "per_student_budget":"Per Student Budget",
                                                                "math_%":"% Passing Math","reading_%":"% Passing Reading",
                                                                "type":"School Type",
                                                                "avg_pass":"% Overall Passing Rate"})

#rename columns
Student_school_summary = Student_school_summary[["School Name", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Reading Score",
                                                "Average Math Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

#create a dataframe for the mapped total budgets and per student budgets
Student_school_summary_mapped = Student_school_summary

Student_school_summary_mapped["Per Student Budget"] = Student_school_summary_mapped["Per Student Budget"].map("${:.2f}".format)
Student_school_summary_mapped["Total School Budget"] = Student_school_summary_mapped["Total School Budget"].map("${:,.2f}".format)



C:\Users\bbeck\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [13]:
#Sort the dataframe developed above to identify the top 5 schools based on overall passing rate
Student_school_summary_Best = Student_school_summary_mapped.sort_values("% Overall Passing Rate", ascending=False)
Student_school_summary_Best.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [14]:
#Sort the dataframe developed above to identify the bottom 5 schools based on overall passing rate
Student_school_summary_Worst = Student_school_summary_mapped.sort_values("% Overall Passing Rate", ascending=True)
Student_school_summary_Worst.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
#Create list of grades
grades = ["9th", "10th", "11th", "12th"]

#calculate the average scores by grade using conditional statements
nineth = school_data_complete.where(school_data_complete['grade'] == '9th')
#remove all NaN
nineth = nineth.dropna(how="any")
#Group by school_name
nineth = nineth.groupby('school_name').mean()

#same process as above, but for 10th grade. I am working on developing for loop 
tenth = school_data_complete.where(school_data_complete['grade'] == '10th')
tenth = nineth.dropna(how="any")
tenth = nineth.groupby('school_name').mean()

#same same as last section
eleventh = school_data_complete.where(school_data_complete['grade'] == '11th')
eleventh = nineth.dropna(how="any")
eleventh = nineth.groupby('school_name').mean()

#same as last section
twelveth = school_data_complete.where(school_data_complete['grade'] == '12th')
twelveth = nineth.dropna(how="any")
twelveth = nineth.groupby('school_name').mean()

#merging dataframes
nine_ten = pd.merge(nineth, tenth, how="outer", on="school_name")
nine_ten = nine_ten[["reading_score_x", "math_score_x", "reading_score_y", "math_score_y"]]
nine_ten = nine_ten.rename(columns={"reading_score_x":"reading_score_9th", 
                                                                "math_score_x":"math_score_9th",
                                                                "reading_score_y":"reading_score_10th",
                                                                "math_score_y":"math_score_10th"})
#Merging dataframes
nine_ten_eleven = pd.merge(nine_ten, eleventh, how="outer", on="school_name")
#Merging dataframes
all_grades = pd.merge(nine_ten_eleven, twelveth, how="outer", on="school_name")
#Renaming columns 
all_grades = all_grades[["reading_score_9th", "math_score_9th", "reading_score_10th", "math_score_10th",
                     "reading_score_x", "math_score_x", "reading_score_y", "math_score_y"]]
#renaming columns
all_grades = all_grades.rename(columns={"reading_score_x":"reading_score_11th", 
                                    "math_score_x":"math_score_11th",
                                    "reading_score_y":"reading_score_12th",
                                    "math_score_y":"math_score_12th"})

In [30]:
#Grabbing only reading scores for each grade
reading_grades = all_grades[["reading_score_9th", "reading_score_10th", "reading_score_11th", "reading_score_12th"]]


math_grades = all_grades[["math_score_9th", "math_score_10th", "math_score_11th", "math_score_12th"]]

#renaming math Grades
math_grades = math_grades.rename(columns={"school_name":"",
                                          "math_score_9th":"9th", 
                                          "math_score_10th":"10th",
                                          "math_score_11th":"11th",
                                          "math_score_12th":"12th"})
#print math grades
math_grades

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [16]:
#Renaming columns
reading_grades = reading_grades.rename(columns={"school_name":"",
                                                "reading_score_9th":"9th", 
                                                "reading_score_10th":"10th",
                                                "reading_score_11th":"11th",
                                                "reading_score_12th":"12th"})
#printing reading grade table by grade
reading_grades

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [31]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#merging the bins and the dataframe
Student_school_summary["Spending Ranges (Per Student)"] = pd.cut(Student_school_summary["Per Student Budget"], spending_bins, labels=group_names)
school_spending = Student_school_summary.groupby("Spending Ranges (Per Student)")



# Get the average of each column within the GroupBy object
school_spending[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.950355
$585-615,83.599686,83.885211,0.942309,0.959003,0.950656
$615-645,79.079225,81.891436,0.756682,0.861066,0.808874
$645-675,76.997210,81.027843,0.661648,0.811340,0.736494


## Scores by School Size

* Perform the same operations as above, based on school size.

In [38]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#merging the bins and the dataframe
Student_school_summary["School Size"] = pd.cut(Student_school_summary["Total Students"], size_bins, labels=group_names)
school_size = Student_school_summary.groupby("School Size")



# Get the average of each column within the GroupBy object
school_size[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [20]:
#grouping data by school type
school_type_grouped = Student_school_summary.groupby("School Type")
#calculate average by group type and only selection certain columns
school_type_grouped = school_type_grouped[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()
school_type_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
